In [1]:
import flint

Flint has arb and calcium now folded in
https://fredrikj.net/python-flint/

In [11]:
x = flint.arb(mid=7, rad=1e-10)
#dir(x)
flint.arb_poly([x, 1]).integral().derivative()
flint.arb_poly.interpolate([0, 1], [0, 1])

1.00000000000000*x

Interval analysis is a computational constructive framing

Moore Book
Bridger book
Errett Bishop

Kaucher intervals https://arxiv.org/pdf/2202.03058

Fixed point theorems.
An uninterpreted sort is not that bad for non recursive datatypes.

In [ ]:
from z3 import *

I = DeclareSort("Interval")
a, b, c = Consts("a b c", I)
R = RealSort()
x,y,z = Reals("x y z")
low = Function("low", I, R)
high = Function("high", I, R)
ForAll([a], low(a) <= high(a))

sub = Function("sub", I, I, BoolSort())
sub_def = ForAll([a,b], sub(a,b) == And(low(a) > low(b), high(a) < high(b)))

sub_irrefl = ForAll([a], Not(sub(a,a)))
prove(Implies(sub_def, sub_irrefl))

sub_asym = ForAll([a,b], Implies(sub(a,b), Not(sub(b,a))))
prove(Implies(sub_def, sub_asym))

sub_trans = ForAll([a,b,c], Implies(And(sub(a,b), sub(b,c)), sub(a,c)))
prove(Implies(sub_def, sub_trans))

subeq = Function("subeq", I, I, BoolSort())
subeq_def = ForAll([a,b], subeq(a,b) == Or(sub(a,b), a == b))

subeq_refl = ForAll([a], subeq(a,a))
prove(Implies(subeq_def, subeq_refl))

subeq_asym = ForAll([a,b], Implies(And(subeq(a,b), subeq(b,a)), a == b))
prove(Implies(And(subeq_def, sub_asym), subeq_asym))

subeq_trans = ForAll([a,b,c], Implies(And(subeq(a,b), subeq(b,c)), subeq(a,c)))
prove(Implies(And(subeq_def, sub_trans), subeq_trans))

# An interval can be dominated by another interval. This is distinct from subset
lt = Function("lt", I, I, BoolSort())
lt_def = ForAll([a,b], lt(a,b) == high(a) < low(b))


mono = Function("mono", ArraySort(I,I), BoolSort())
f = Array("f", I, I)
mono_def = ForAll([f], mono(f) == ForAll([a], sub(a,f[a])))

fix = Function("fix", ArraySort(I,I), I)
fix_def = ForAll([f], Implies(mono(f), fix(f) == f[fix(f)]))

# too dangerous to skolemize?
int_exists = ForAll([x,y], Implies( x <= y, Exists([a], And(low(a) == x, high(a) == y))))

iadd = Function("iadd", I, I, I)

iadd_comm = ForAll([x,y], iadd(x,y) == iadd(y,x))
iadd_assoc = ForAll([x,y,z], iadd(x,iadd(y,z)) == iadd(iadd(x,y),z))
iadd_mono = ForAll([x,y,z], Implies(sub(x,y), sub(iadd(x,z),iadd(y,z))))

iadd_high = ForAll([a,b], high(iadd(a,b)) == high(a) + high(b))
iadd_low = ForAll([a,b], low(iadd(a,b)) == low(a) + low(b))

iadd_monol = ForAll([a], mono(Lambda[a], iadd(a,b)))